In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt #to plot the graph

# model, layers, optimizer
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras import optimizers
from keras import regularizers
from keras.layers.convolutional import Convolution2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils.np_utils import to_categorical

# scikit-learn library 
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from sklearn.model_selection import train_test_split

# for session
from keras import backend as K

In [ ]:
# loading the given dataset of compition
train = pd.read_csv("Digit_reco/train.csv").values.astype('float32')
test  = pd.read_csv("Digit_reco/test.csv").values.astype('float32')

In [ ]:
# spliting the training data-set  
x_train, x_test, y_train, y_test= train_test_split(train[:,1:],train[:,0],test_size=0.2)

In [ ]:
# reshaping the data into required dimension
x_train = x_train.reshape(-1,28,28,1)
x_test = x_test.reshape(-1,28,28,1)

train_labels = to_categorical(y_train, 10)
test_labels = to_categorical(y_test, 10)

In [ ]:
# to display the sample 
def display_sample(num):
    #Print the one-hot array of this sample's label 
    print(train_labels[num])  
    #Print the label converted back to a number
    label = train_labels[num].argmax(axis=0)
    #Reshape the 768 values to a 28x28 image
    image = x_train[num].reshape([28,28])
    plt.title('Sample: %d  Label: %d' % (num, label))
    plt.imshow(image, cmap=plt.get_cmap('gray_r'))
    plt.show()
    
display_sample(100)

In [ ]:
# model defination

model = Sequential()

# 3x3 and 6x6 kernels
model.add(Convolution2D(filters=20, kernel_size=(6,6), kernel_regularizer=regularizers.l2(0.04), strides=(1,1), padding='valid', activation='relu', data_format='channels_last', input_shape=(28,28,1)))
model.add(Convolution2D(filters=20, kernel_size=(3,3), kernel_regularizer=regularizers.l2(0.04), strides=(1,1), padding='valid', activation='relu'))
# Reduce by taking the max of each 4x4 block
model.add(MaxPooling2D(pool_size=(4,4), strides=(1,1)))
# Dropout to avoid overfitting
model.add(Dropout(rate=0.05,seed=3))

model.add(Convolution2D(filters=10, kernel_size=(6,6), kernel_regularizer=regularizers.l2(0.04), strides=(1,1), padding='valid', activation='relu'))
model.add(Convolution2D(filters=10, kernel_size=(3,3), kernel_regularizer=regularizers.l2(0.04), strides=(1,1), padding='valid', activation='relu'))
model.add(MaxPooling2D(pool_size=(4,4), strides=(1,1)))
model.add(Dropout(rate=0.05,seed=8))

# Flatten the results to one dimension for passing into our final layer
model.add(Flatten())

model.add(Dense(units=30, activation='tanh', kernel_regularizer=regularizers.l2(0.04)))
# Final categorization from 0-9 with softmax
model.add(Dense(units=10, activation='softmax', kernel_regularizer=regularizers.l2(0.04)))

# reduce the learning rate if training accuracy suddenly drops and keeps decreasing
sgd = optimizers.SGD(lr=0.003) # lr by default is 0.01 for SGD


In [ ]:
# model summary
model.summary()

In [ ]:
from keras import metrics

In [ ]:
# model compilation
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=[metrics.categorical_accuracy])

In [ ]:
# model fitting
history = model.fit(x_train, train_labels,
                    batch_size=50,
                    epochs=5,
                    verbose=2,
                    validation_split=0.1)

In [ ]:
# prediction on given test data-set
given_test = test
given_test = given_test.reshape(-1,28,28,1)/255.
predict = model.predict_classes(given_test)

In [ ]:
# saving the model inti Json file
model_json = model.to_json()
open('digit_recognizer_model_using_cnn_KG.json','w').write(model_json)
model.save_weights('digit_recognizer_weights_model_usin_cnn_KG.h5')

In [ ]:
#saving the output
def write_predict(predict,name):
    pd.DataFrame({'ImageId':list(range(1,len(predict)+1)),'Label':predict}).to_csv(name,index=False,header=True)
    
write_predict(predict,'keras_digit_recognizer_by_cnn_output_KG.csv')

# Write-up

In [ ]:
#next file 